In [1]:
import pyterrier as pt

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from experiments_helper import load_sparse_index_from_disk

dataset_name = "nfcorpus"

retriever = load_sparse_index_from_disk(dataset_name)

In [3]:
from experiments_helper import run_experiment

dataset_test_name = "irds:beir/nfcorpus/test"
run_experiment(retriever, dataset_test_name)

,name,RR@10,nDCG@10,AP@100
0,BR(BM25),0.534378,0.322219,0.143582


Create the query encoder that will run on CPU


In [3]:
from gte_base_en_encoder import GTEBaseDocumentEncoder
q_encoder = GTEBaseDocumentEncoder("Alibaba-NLP/gte-base-en-v1.5")


Retrieve the dense index(already loaded into memory)

In [4]:
from experiments_helper import load_dense_index_from_disk
dense_index = load_dense_index_from_disk(dataset_name, q_encoder)

100%|██████████| 3633/3633 [00:00<00:00, 173784.05it/s]


In [7]:
from fast_forward.util.pyterrier import FFScore

from fast_forward.util.pyterrier import FFInterpolate

ff_score = FFScore(dense_index)
ff_int = FFInterpolate(alpha=0.05)

In [9]:
from experiments_helper import run_experiment

dataset_test_name = "irds:beir/nfcorpus/test"

pipeline = retriever  % 1000  >> ff_score >> ff_int
run_experiment(pipeline, dataset_test_name)

,name,RR@10,nDCG@10,AP@100
0,"Compose(Compose(RankCutoff(BR(BM25), 1000), FF...",0.553675,0.343964,0.15507
